In [1]:
import random
import time
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import clear_output

import fastlob as lob

In [2]:
def generate_orders(T: int, midprice: float):
    result = list()
    
    for _ in range(T):
    
        n_ask_limits = stats.poisson.rvs(500)
        n_bid_limits = stats.poisson.rvs(500)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [lob.OrderParams(lob.OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [lob.OrderParams(lob.OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((lob.OrderSide.BID, lob.OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [lob.OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [3]:
def simulate(orders: list, speed: float):
    ob = lob.Orderbook('Simulation')
    ob.start()

    for o in orders:
        ob.process_many(o)
        ob.render()
        time.sleep(speed)
        clear_output(wait=True)
        
    ob.stop()

In [4]:
midprice = stats.uniform.rvs(0.1, 1000)
orders = generate_orders(1000, midprice)

In [5]:
simulate(orders, 0.1)

   [ORDER-BOOK Simulation]

   ...(899 more asks)
 - 861.16 | 4544 | 233951.43 | 201469613.4588
 - 861.15 | 4530 | 233119.70 | 200751029.6550
 - 861.14 | 4738 | 239813.78 | 206513238.5092
 - 861.13 | 4684 | 241234.01 | 207733843.0313
 - 861.12 | 4666 | 241042.01 | 207566095.6512
 - 861.11 | 4725 | 243514.84 | 209693063.8724
 - 861.10 | 812 | 40278.06 | 34683437.4660
 - 861.09 | 029 | 01522.60 | 1311095.6340
 - 861.08 | 002 | 00071.20 | 61308.8960
 - 861.07 | 002 | 00141.42 | 121772.5194
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 - 861.05 | 001 | 00034.33 | 29559.8465
 - 861.04 | 002 | 00106.83 | 91984.9032
 - 861.03 | 006 | 00214.56 | 184742.5968
 - 861.02 | 002 | 00042.96 | 36989.4192
 - 861.01 | 013 | 00430.38 | 370561.4838
 - 861.00 | 4513 | 227967.57 | 196280077.7700
 - 860.99 | 4855 | 245229.83 | 211140431.3317
 - 860.98 | 4789 | 240131.98 | 206748832.1404
 - 860.97 | 4787 | 244269.32 | 210308556.4404
 - 860.96 | 4679 | 236946.46 | 204001424.2016
   ...(895 more bids)

    spread = 